In [2]:
import requests
import pandas as pd

# Set the API URL and token
url = "https://api.hamsterkombatgame.io/clicker/upgrades-for-buy"
token = "1722874841725QW7tl9Z5BUnql4Spw51xuoFUPpD53zDnKWj3sk6xI2eWfi73jHh0wHErXONEkiXO6479354365"

# Function to fetch upgrade data
def fetch_upgrades():
    headers = {
        "accept": "*/*",
        "accept-language": "en-US,en;q=0.9,hi;q=0.8",
        "authorization": f"Bearer {token}",
        "origin": "https://hamsterkombatgame.io",
        "referer": "https://hamsterkombatgame.io/",
        "sec-ch-ua": '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-site",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
    }

    response = requests.post(url, headers=headers)
    response.raise_for_status()

    upgrades = response.json()["upgradesForBuy"]

    # Filter and process upgrades
    valid_upgrades = [
        {
            **upgrade,
            "priceToProfitRatio": upgrade["price"] / upgrade["profitPerHourDelta"]
        }
        for upgrade in upgrades
        if not upgrade["isExpired"]
        and upgrade["price"] > 0
        and upgrade["profitPerHourDelta"] > 0
        and upgrade["profitPerHour"] > 0
        and (upgrade.get("cooldownSeconds") is None or upgrade["cooldownSeconds"] == 0)
    ]

    return valid_upgrades

# Fetch the upgrades
upgrades = fetch_upgrades()

# Create a DataFrame for analysis
df_upgrades = pd.DataFrame(upgrades)

# Sort upgrades by price-to-profit ratio
df_sorted = df_upgrades.sort_values(by="priceToProfitRatio")

# Display available upgrades in a table
display_columns = ["name", "section", "level", "price", "profitPerHourDelta", "priceToProfitRatio"]
print("Available Upgrades:")
print(df_sorted[display_columns].to_string(index=False))

# Function to find the best combination
def find_best_combination(budget, desired_profit):
    valid_upgrades = df_sorted[df_sorted["price"] <= budget]
    best_combination = []
    max_profit = 0

    # Check single upgrades first
    for _, upgrade in valid_upgrades.iterrows():
        if upgrade["profitPerHourDelta"] >= desired_profit:
            if len(best_combination) == 0 or upgrade["profitPerHourDelta"] > max_profit:
                best_combination = [upgrade]
                max_profit = upgrade["profitPerHourDelta"]

    # Check combinations of upgrades
    for i in range(len(valid_upgrades)):
        combination = []
        current_profit = 0
        current_price = 0

        for j in range(i, len(valid_upgrades)):
            if current_price + valid_upgrades.iloc[j]["price"] <= budget:
                combination.append(valid_upgrades.iloc[j])
                current_profit += valid_upgrades.iloc[j]["profitPerHourDelta"]
                current_price += valid_upgrades.iloc[j]["price"]

        if current_profit >= desired_profit and current_profit > max_profit:
            best_combination = combination
            max_profit = current_profit

    return best_combination

# User input for budget and desired profit
budget = float(input("\nEnter your budget: "))
desired_profit = float(input("Enter your desired profit increase: "))

# Find best options
best_options = find_best_combination(budget, desired_profit)

# Display results
if best_options:
    best_options_df = pd.DataFrame(best_options)
    total_price = best_options_df["price"].sum()
    total_profit = best_options_df["profitPerHourDelta"].sum()

    print("\nBest Options:")
    print(best_options_df[["name", "profitPerHourDelta", "price"]].to_string(index=False))
    print(f"\nTotal Price for Best Options: {total_price}")
    print(f"Total Profit Increase for Best Options: {total_profit}")
else:
    print("\nNo available cards satisfy the budget and desired profit increase.")


Available Upgrades:
                          name  section  level    price  profitPerHourDelta  priceToProfitRatio
                   HamsterBank Specials     12 12843911               39992          321.162007
               CX Hub Istanbul Specials     12  2354717                6315          372.876801
             Data Сenter Tokyo  PR&Team     11  4767980               12786          372.906304
                 NFT Metaverse     Web3      5   989966                2622          377.561404
                Crypto Farming     Web3     10  2090804                5515          379.112239
          50M Telegram channel Specials     10   696935                1838          379.181175
         Anti money loundering    Legal     13   242191                 631          383.820919
                   P2P trading  Markets     13   339068                 878          386.182232
    Anonymous transactions ban  PR&Team     15   299067                 774          386.391473
           HamsterSt